In [22]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta


# ---------- Helper: date range ----------
def date_range(days):
    start = datetime(2024, 1, 1)
    return [start + timedelta(days=i) for i in range(days)]


### Generate Data 

1) Random Walk -- Start from an initial price and at each time step add a random change (positive or negative).
2) Geometric Brownian Motion (GBM) -- Similar to random walk but changes are proportional to the current price, simulating percentage changes., doesnt goes negative. this is used in finance 
3) Autoregressive Model (AR(1)) -- Each price depends on the previous price plus some noise, introducing correlation over time.
4) Jump Process -- Prices follow a random walk but occasionally make large jumps, simulating sudden market movements.

In [23]:
# ---------- Assign Custom starting prices ----------
def assign_start_prices(n_stocks):
    """
    for different starting prices
    3 stocks at 100,
    4 stocks randomly between 50–99,
    3 stocks randomly between 101–150
    """
    starts = []
    # exactly 3 at 100
    starts.extend([100] * 3)
    # 4 below 100
    starts.extend(np.random.randint(50, 100, size=4))
    # 3 above 100
    starts.extend(np.random.randint(101, 151, size=3))
    # shuffle so that stocks are not grouped
    np.random.shuffle(starts)
    return starts

In [24]:
# ---------- Core generator ----------
def generate_market(method="random_walk", n_stocks=10, days=180, start_price=100):
    """
    method : 'random_walk', 'gbm', 'ar1', 'jump'
    Returns a DataFrame with columns: Stock, Date, Price
    """
    dates = date_range(days) # for constant price100
    all_rows = []
    # start_prices = assign_start_prices(n_stocks)

    for s in range(1, n_stocks + 1):
        stock = f"Stock_{s}"
        prices = [start_price]

        if method == "random_walk":
            for _ in range(1, days):
                change = np.random.normal(0, 1)  # small daily change
                prices.append(prices[-1] + change)

        elif method == "gbm":
            mu, sigma = 0.0005, 0.01  # drift & volatility
            for _ in range(1, days):
                z = np.random.normal()
                prices.append(prices[-1] * np.exp((mu - 0.5 * sigma**2) + sigma * z))

        elif method == "ar1":
            alpha, phi = 0, 0.95
            for _ in range(1, days):
                noise = np.random.normal(0, 1)
                next_p = alpha + phi * prices[-1] + noise
                prices.append(next_p)

        elif method == "jump":
            for _ in range(1, days):
                change = np.random.normal(0, 1)
                if np.random.rand() < 0.05:  # 5% chance of a jump
                    change += np.random.normal(0, 10)  # big shock
                prices.append(prices[-1] + change)

        # collect rows
        for d, p in zip(dates, prices):
            all_rows.append([stock, d, p])

    df = pd.DataFrame(all_rows, columns=["Stock", "Date", "Price"])
    return df


In [25]:
# ---------- Visualization ----------
def plot_market(df, title):
    fig = px.line(
        df,
        x="Date",
        y="Price",
        color="Stock",
        title=title,
        labels={"Price": "Stock Price"},
    )
    fig.update_layout(template="plotly_white", hovermode="x unified")
    fig.show()

In [26]:
# ---------- Run all four approaches ----------
methods = ["random_walk", "gbm", "ar1", "jump"]
for m in methods:
    print(f"\n=== {m.upper()} ===")
    data = generate_market(method=m)
    print(data.head(20))  # print first 20 rows of generated data
    plot_market(data, title=f"Synthetic Market Prices – {m.upper()}")



=== RANDOM_WALK ===
      Stock       Date       Price
0   Stock_1 2024-01-01  100.000000
1   Stock_1 2024-01-02   99.260657
2   Stock_1 2024-01-03   98.503831
3   Stock_1 2024-01-04  100.460614
4   Stock_1 2024-01-05  100.012400
5   Stock_1 2024-01-06  100.279673
6   Stock_1 2024-01-07   98.592775
7   Stock_1 2024-01-08   99.045592
8   Stock_1 2024-01-09   98.671510
9   Stock_1 2024-01-10   99.119521
10  Stock_1 2024-01-11   98.995341
11  Stock_1 2024-01-12   99.952548
12  Stock_1 2024-01-13  100.820213
13  Stock_1 2024-01-14  100.951789
14  Stock_1 2024-01-15  100.262268
15  Stock_1 2024-01-16   98.310192
16  Stock_1 2024-01-17   97.920893
17  Stock_1 2024-01-18   98.628659
18  Stock_1 2024-01-19   98.462385
19  Stock_1 2024-01-20   98.635538



=== GBM ===
      Stock       Date       Price
0   Stock_1 2024-01-01  100.000000
1   Stock_1 2024-01-02  100.754696
2   Stock_1 2024-01-03  101.760984
3   Stock_1 2024-01-04  102.581266
4   Stock_1 2024-01-05  102.628335
5   Stock_1 2024-01-06  102.998305
6   Stock_1 2024-01-07  104.026100
7   Stock_1 2024-01-08  103.055449
8   Stock_1 2024-01-09  103.342891
9   Stock_1 2024-01-10  101.272803
10  Stock_1 2024-01-11  101.130009
11  Stock_1 2024-01-12   98.813092
12  Stock_1 2024-01-13   98.134837
13  Stock_1 2024-01-14   96.437885
14  Stock_1 2024-01-15   97.508762
15  Stock_1 2024-01-16   96.939757
16  Stock_1 2024-01-17   96.002265
17  Stock_1 2024-01-18   96.262514
18  Stock_1 2024-01-19   95.593887
19  Stock_1 2024-01-20   94.419340



=== AR1 ===
      Stock       Date       Price
0   Stock_1 2024-01-01  100.000000
1   Stock_1 2024-01-02   94.070794
2   Stock_1 2024-01-03   89.629097
3   Stock_1 2024-01-04   85.838693
4   Stock_1 2024-01-05   81.522157
5   Stock_1 2024-01-06   78.094282
6   Stock_1 2024-01-07   73.594295
7   Stock_1 2024-01-08   71.327133
8   Stock_1 2024-01-09   68.243399
9   Stock_1 2024-01-10   63.590674
10  Stock_1 2024-01-11   61.114346
11  Stock_1 2024-01-12   58.033045
12  Stock_1 2024-01-13   54.225702
13  Stock_1 2024-01-14   50.150434
14  Stock_1 2024-01-15   45.867278
15  Stock_1 2024-01-16   45.032349
16  Stock_1 2024-01-17   43.212247
17  Stock_1 2024-01-18   40.872938
18  Stock_1 2024-01-19   40.594250
19  Stock_1 2024-01-20   41.023474



=== JUMP ===
      Stock       Date       Price
0   Stock_1 2024-01-01  100.000000
1   Stock_1 2024-01-02   99.689270
2   Stock_1 2024-01-03   98.912909
3   Stock_1 2024-01-04   98.771924
4   Stock_1 2024-01-05   99.558369
5   Stock_1 2024-01-06   98.701910
6   Stock_1 2024-01-07   96.938908
7   Stock_1 2024-01-08   96.715825
8   Stock_1 2024-01-09   97.624793
9   Stock_1 2024-01-10   98.309376
10  Stock_1 2024-01-11   97.569025
11  Stock_1 2024-01-12   97.156875
12  Stock_1 2024-01-13   97.068059
13  Stock_1 2024-01-14   96.288602
14  Stock_1 2024-01-15   96.543391
15  Stock_1 2024-01-16   98.457739
16  Stock_1 2024-01-17   99.138795
17  Stock_1 2024-01-18   98.408459
18  Stock_1 2024-01-19   99.794751
19  Stock_1 2024-01-20  109.119948


In [27]:
def plot_all_markets(df,method):
    """
    One Plotly chart per market – shows all 10 stocks in each market.
    """
    fig = px.line(
        df,
        x="Date",
        y="Price",
        color="Stock",
        facet_col="Market",  # separate panel for each market
        facet_col_wrap=2,  # wrap for readability
        title=f"Synthetic Prices – {method.upper()} Method",
        labels={"Price": "Stock Price"},
    )
    fig.update_layout(template="plotly_white", hovermode="x unified")
    fig.show()
    

In [28]:
def generate_market(market_name, method="random_walk", n_stocks=10, days=180, start_price=100):
    """
    Generate synthetic stock price data for ONE market
    method : 'random_walk', 'gbm', 'ar1', 'jump'
    """
    dates = date_range(days)
    all_rows = []
    # start_prices = assign_start_prices(n_stocks)

    for s in range(1, n_stocks + 1):
        stock = f"Stock_{s}"
        # prices = [start_prices[s - 1]]
        prices=[start_price]

        if method == "random_walk":
            for _ in range(1, days):
                change = np.random.normal(0, 1)
                prices.append(prices[-1] + change)

        elif method == "gbm":
            mu, sigma = 0.0005, 0.01
            for _ in range(1, days):
                z = np.random.normal()
                prices.append(prices[-1] * np.exp((mu - 0.5 * sigma**2) + sigma * z))

        elif method == "ar1":
            alpha, phi = 0, 0.95
            for _ in range(1, days):
                noise = np.random.normal(0, 1)
                prices.append(alpha + phi * prices[-1] + noise)

        elif method == "jump":
            for _ in range(1, days):
                change = np.random.normal(0, 1)
                if np.random.rand() < 0.05:  # 5% chance of a big jump
                    change += np.random.normal(0, 10)
                prices.append(prices[-1] + change)

        for d, p in zip(dates, prices):
            all_rows.append([market_name, stock, d, p])

    return pd.DataFrame(all_rows, columns=["Market", "Stock", "Date", "Price"])


In [29]:
methods = ["random_walk", "gbm", "ar1", "jump"]  # 4 approaches
markets = [f"Market_{i}" for i in range(1, 6)]  # 5 markets

for m in methods:
    print(f"\n==== {m.upper()} ====")
    all_data = pd.DataFrame()

    for mk in markets:
        mk_data = generate_market(market_name=mk, method=m)
        all_data = pd.concat([all_data, mk_data], ignore_index=True)

    # show first few rows of generated synthetic market data
    print(all_data.head(20))

    # Visualize
    plot_all_markets(all_data, method=m)


==== RANDOM_WALK ====
      Market    Stock       Date       Price
0   Market_1  Stock_1 2024-01-01  100.000000
1   Market_1  Stock_1 2024-01-02   98.086321
2   Market_1  Stock_1 2024-01-03   96.459308
3   Market_1  Stock_1 2024-01-04   97.426304
4   Market_1  Stock_1 2024-01-05   96.902187
5   Market_1  Stock_1 2024-01-06   97.253166
6   Market_1  Stock_1 2024-01-07   96.126906
7   Market_1  Stock_1 2024-01-08   97.760302
8   Market_1  Stock_1 2024-01-09   97.139587
9   Market_1  Stock_1 2024-01-10   96.935567
10  Market_1  Stock_1 2024-01-11   96.906731
11  Market_1  Stock_1 2024-01-12   97.936799
12  Market_1  Stock_1 2024-01-13   98.922542
13  Market_1  Stock_1 2024-01-14   98.984951
14  Market_1  Stock_1 2024-01-15  100.627107
15  Market_1  Stock_1 2024-01-16  101.634100
16  Market_1  Stock_1 2024-01-17  100.927719
17  Market_1  Stock_1 2024-01-18  100.095743
18  Market_1  Stock_1 2024-01-19   99.446037
19  Market_1  Stock_1 2024-01-20   98.538747



==== GBM ====
      Market    Stock       Date       Price
0   Market_1  Stock_1 2024-01-01  100.000000
1   Market_1  Stock_1 2024-01-02   99.781862
2   Market_1  Stock_1 2024-01-03  100.822684
3   Market_1  Stock_1 2024-01-04  101.452047
4   Market_1  Stock_1 2024-01-05  101.258768
5   Market_1  Stock_1 2024-01-06   99.821250
6   Market_1  Stock_1 2024-01-07   99.338197
7   Market_1  Stock_1 2024-01-08  100.961657
8   Market_1  Stock_1 2024-01-09  102.768904
9   Market_1  Stock_1 2024-01-10  102.920662
10  Market_1  Stock_1 2024-01-11  102.495075
11  Market_1  Stock_1 2024-01-12  102.370838
12  Market_1  Stock_1 2024-01-13  102.775236
13  Market_1  Stock_1 2024-01-14  102.420616
14  Market_1  Stock_1 2024-01-15  101.380497
15  Market_1  Stock_1 2024-01-16  101.532774
16  Market_1  Stock_1 2024-01-17  101.333491
17  Market_1  Stock_1 2024-01-18  102.084147
18  Market_1  Stock_1 2024-01-19  100.852483
19  Market_1  Stock_1 2024-01-20  101.143781



==== AR1 ====
      Market    Stock       Date       Price
0   Market_1  Stock_1 2024-01-01  100.000000
1   Market_1  Stock_1 2024-01-02   93.777991
2   Market_1  Stock_1 2024-01-03   89.069191
3   Market_1  Stock_1 2024-01-04   86.236610
4   Market_1  Stock_1 2024-01-05   80.473458
5   Market_1  Stock_1 2024-01-06   76.463064
6   Market_1  Stock_1 2024-01-07   72.997298
7   Market_1  Stock_1 2024-01-08   68.281653
8   Market_1  Stock_1 2024-01-09   66.750621
9   Market_1  Stock_1 2024-01-10   65.471138
10  Market_1  Stock_1 2024-01-11   61.815513
11  Market_1  Stock_1 2024-01-12   57.367572
12  Market_1  Stock_1 2024-01-13   53.898811
13  Market_1  Stock_1 2024-01-14   51.137230
14  Market_1  Stock_1 2024-01-15   48.521910
15  Market_1  Stock_1 2024-01-16   48.800944
16  Market_1  Stock_1 2024-01-17   45.383023
17  Market_1  Stock_1 2024-01-18   43.532950
18  Market_1  Stock_1 2024-01-19   42.164022
19  Market_1  Stock_1 2024-01-20   41.930315



==== JUMP ====
      Market    Stock       Date       Price
0   Market_1  Stock_1 2024-01-01  100.000000
1   Market_1  Stock_1 2024-01-02  100.200481
2   Market_1  Stock_1 2024-01-03  101.481526
3   Market_1  Stock_1 2024-01-04  103.260776
4   Market_1  Stock_1 2024-01-05  103.209000
5   Market_1  Stock_1 2024-01-06  100.839242
6   Market_1  Stock_1 2024-01-07   98.637574
7   Market_1  Stock_1 2024-01-08   98.693844
8   Market_1  Stock_1 2024-01-09   98.431639
9   Market_1  Stock_1 2024-01-10   98.250040
10  Market_1  Stock_1 2024-01-11   98.843709
11  Market_1  Stock_1 2024-01-12   98.932866
12  Market_1  Stock_1 2024-01-13   99.507295
13  Market_1  Stock_1 2024-01-14   98.601846
14  Market_1  Stock_1 2024-01-15  100.192252
15  Market_1  Stock_1 2024-01-16   99.793468
16  Market_1  Stock_1 2024-01-17  101.342171
17  Market_1  Stock_1 2024-01-18  101.688416
18  Market_1  Stock_1 2024-01-19  101.420409
19  Market_1  Stock_1 2024-01-20  102.379121
